Build an AI model that can classify SMS messages as spam or legitimate. Use techniques like TF-IDF or word embeddings with classifiers like Naive Bayes, Logistic Regression, or Support Vector Machines to identify spam messages

In [2]:
# import libraries
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
from sklearn.naive_bayes import MultinomialNB

In [5]:
# import dataset
data = pd.read_csv('/content/spam.csv', encoding='latin-1')

In [6]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [7]:
# data info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [9]:
# Remove blank Columns
data.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)

In [10]:
print(data)

        v1                                                 v2
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham              Will Ì_ b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [18]:
# data info after remove column
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [19]:
# Size of data
data.shape

(5572, 2)

In [13]:
# Attribute of Dataset
data.columns

Index(['v1', 'v2'], dtype='object')

In [15]:
# Description of dataset
data.describe()

,v1,v2
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [17]:
# descriptive statastics for categorical features
data.describe(include= 'object')

,v1,v2
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [20]:
# Drop duplicate values
data.duplicated().sum()

403

In [23]:
data = data.drop_duplicates(keep='first')

In [55]:
# After removing duplicate value
data.shape

(5169, 2)

Label Encoding

In [29]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data['v1'] = encoder.fit_transform(data['v1'])

In [30]:
data['v1'].value_counts()

v1
0    4516
1     653
Name: count, dtype: int64

In [31]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Remove special characters and punctuation and Convert text to lowercase

In [33]:
def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    ps = PorterStemmer()
    stemmed_text = [ps.stem(word) for word in filtered_text]
    return ' '.join(stemmed_text)
data['v2'] = data['v2'].apply(preprocess_text)

In [35]:
# Split the data
input = data['v2']
output = data['v1']
X_train1, X_test1, y_train, y_test = train_test_split(input, output, test_size=0.2, random_state=42)

In [36]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train1)
X_test = vectorizer.transform(X_test1)

In [37]:
X_train

<4135x6354 sparse matrix of type '<class 'numpy.int64'>'
	with 34237 stored elements in Compressed Sparse Row format>

In [38]:
y_train

2228    0
5529    0
2149    0
5058    1
5051    0
       ..
4740    0
474     0
3266    0
4016    0
879     1
Name: v1, Length: 4135, dtype: int64

Naive Bayes

In [39]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [40]:
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

In [41]:
train_accuracy = accuracy_score(y_train, y_pred_train)
train_precision = precision_score(y_train, y_pred_train, average='weighted')
train_recall = recall_score(y_train, y_pred_train, average='weighted')
train_f1 = f1_score(y_train, y_pred_train, average='weighted')
train_confusion = confusion_matrix(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test, average='weighted')
test_recall = recall_score(y_test, y_pred_test, average='weighted')
test_f1 = f1_score(y_test, y_pred_test, average='weighted')
test_confusion = confusion_matrix(y_test, y_pred_test)

print(f"Training Accuracy: {train_accuracy}")
print(f"Training Precision: {train_precision}")
print(f"Training Recall: {train_recall}")
print(f"Training F1 Score: {train_f1}")
print("Training Confusion Matrix:")
print(train_confusion)

Training Accuracy: 0.9934703748488513
Training Precision: 0.9934561329058047
Training Recall: 0.9934703748488513
Training F1 Score: 0.9934620557586109
Training Confusion Matrix:
[[3615   12]
 [  15  493]]


In [42]:
print(f"\nTesting Accuracy: {test_accuracy}")
print(f"Testing Precision: {test_precision}")
print(f"Testing Recall: {test_recall}")
print(f"Testing F1 Score: {test_f1}")
print("Testing Confusion Matrix:")
print(test_confusion)


Testing Accuracy: 0.9825918762088974
Testing Precision: 0.9824192012781614
Testing Recall: 0.9825918762088974
Testing F1 Score: 0.9824372613280711
Testing Confusion Matrix:
[[883   6]
 [ 12 133]]


Decision Tree

In [44]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz
param_grid = {
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

dt_classifier = DecisionTreeClassifier(random_state=4)
grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_estimator_

Fitting 5 folds for each of 36 candidates, totalling 180 fits


In [45]:
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)
best_dt_regressor = DecisionTreeClassifier(**best_params)
best_dt_regressor.fit(X_train, y_train)

Best Hyperparameters: {'max_depth': 9, 'min_samples_leaf': 4, 'min_samples_split': 2}


DecisionTreeClassifier(max_depth=9, min_samples_leaf=4)

In [46]:
y_pred_train = best_dt_regressor.predict(X_train)
y_pred_test = best_dt_regressor.predict(X_test)

In [47]:
train_accuracy = accuracy_score(y_train, y_pred_train)
train_precision = precision_score(y_train, y_pred_train, average='weighted')
train_recall = recall_score(y_train, y_pred_train, average='weighted')
train_f1 = f1_score(y_train, y_pred_train, average='weighted')
train_confusion = confusion_matrix(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test, average='weighted')
test_recall = recall_score(y_test, y_pred_test, average='weighted')
test_f1 = f1_score(y_test, y_pred_test, average='weighted')
test_confusion = confusion_matrix(y_test, y_pred_test)

print(f"Training Accuracy: {train_accuracy}")
print(f"Training Precision: {train_precision}")
print(f"Training Recall: {train_recall}")
print(f"Training F1 Score: {train_f1}")
print("Training Confusion Matrix:")
print(train_confusion)

Training Accuracy: 0.96590084643289
Training Precision: 0.9654426302811969
Training Recall: 0.96590084643289
Training F1 Score: 0.9642897151456332
Training Confusion Matrix:
[[3606   21]
 [ 120  388]]


In [48]:
print(f"\nTesting Accuracy: {test_accuracy}")
print(f"Testing Precision: {test_precision}")
print(f"Testing Recall: {test_recall}")
print(f"Testing F1 Score: {test_f1}")
print("Testing Confusion Matrix:")
print(test_confusion)


Testing Accuracy: 0.9574468085106383
Testing Precision: 0.956848885538015
Testing Recall: 0.9574468085106383
Testing F1 Score: 0.9553448575525398
Testing Confusion Matrix:
[[882   7]
 [ 37 108]]


Logistic Regression

In [49]:
from sklearn.linear_model import LogisticRegression

# logistic regression model
logistic_regression = LogisticRegression(random_state=42)

# hyperparameter grid
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(logistic_regression, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the logistic regression model with the best hyperparameters
best_logistic_regression = LogisticRegression(**best_params)
best_logistic_regression.fit(X_train, y_train)

# Evaluate the logistic regression model on the test set
y_pred_test = best_logistic_regression.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Print the test accuracy
print(f"Test accuracy: {test_accuracy}")


Fitting 5 folds for each of 120 candidates, totalling 600 fits
Test accuracy: 0.9787234042553191


Support Vector Machine

In [50]:
from sklearn.svm import SVC

# SVM classifier
svm_classifier = SVC()

# hyperparameter grid
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto']
}

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(svm_classifier, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the SVM model with the best hyperparameters
best_svm_classifier = SVC(**best_params)
best_svm_classifier.fit(X_train, y_train)

# Evaluate the SVM model on the test set
y_pred_test = best_svm_classifier.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Print the test accuracy
print(f"Test accuracy: {test_accuracy}")

# Print the best hyperparameters
print(f"Best hyperparameters: {best_params}")


Fitting 5 folds for each of 32 candidates, totalling 160 fits
Test accuracy: 0.9845261121856866
Best hyperparameters: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}


In [53]:
svm = SVC(C = 1, gamma = 'scale', kernel = 'linear')
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
print("\nSupport Vector Machine:")
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))
print(confusion_matrix(y_test, y_pred_svm))


Support Vector Machine:
Accuracy: 0.9845261121856866
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       889
           1       1.00      0.89      0.94       145

    accuracy                           0.98      1034
   macro avg       0.99      0.94      0.97      1034
weighted avg       0.98      0.98      0.98      1034

[[889   0]
 [ 16 129]]


In [54]:
models = ['Naive Bayes', 'Decision Tree', 'Logistic Regression', 'SVM']
accuracy_scores = [test_accuracy, test_accuracy, test_accuracy, test_accuracy]

# Create a table
table = pd.DataFrame({'Model': models, 'Accuracy': accuracy_scores})

# Display the table
print(table.to_string())


                 Model  Accuracy
0          Naive Bayes  0.984526
1        Decision Tree  0.984526
2  Logistic Regression  0.984526
3                  SVM  0.984526
